# Diffusion simulation classes

> Ricardo Peres, 06.05.2022

In [1]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
import scipy.stats
from scipy.optimize import curve_fit
import scipy.integrate
from mpl_toolkits.mplot3d import Axes3D


#non-trivial packages
from hexalattice.hexalattice import create_hex_grid

In [13]:
class TPC:
    '''
    General TPC class. Contains useful constants, common variables 
    and functions relatesd to the overall system.
    '''
    
    def __init__(self, radius, length, gas_gap, drift_field):
        self.radius = radius #
        self.length = length #lenght of liquid bellow gate
        self.gas_gap = gas_gap #distance from liquid-gas interface to sensors
        self.drift_field = drift_field
        self.drift_velocity = self.model_velocity(self.drift_field)
        self.diffusion_long = self.model_diffusion_longitudinal(self.drift_field)
        self.diffusion_trans = self.model_diffusion_transversal(self.drift_field)
        
        self.mesh_pitch = 1.56
    
    @classmethod
    def model_velocity(cls, drift_field):
        '''
        Drift velocity model from NEST, implemented by Yanina 
        Biondi (https://github.com/YaniBion).
        Given a drift_field values returns the expected electron drift 
        velocity in LXe. In mm/us
        '''
        #par = [-3.1046, 27.037, -2.1668, 193.27, -4.8024, 646.04, 9.2471]
        par = [-1.5000, 28.510, -.21948, 183.49, -1.4320, 1652.9, 2.884]
        dv = (par[0] * np.exp(-drift_field/par[1]) + 
              par[2] * np.exp(-drift_field/par[3]) + 
              par[4] * np.exp(-drift_field/par[5]) +
              par[6]) #mm/us
        return dv
    
    @classmethod
    def model_diffusion_longitudinal(cls, drift_field):
        '''
        Longitudinal diffusion model: TO DO.
        For 100 V/cm -> ~26cm2/us from 1T
        '''
        if drift_field != 100:
            warnings.warn('Only the long. diffusino value for 100 V/cm is implemented, going with it for now.')
        D_l = 26 #cm2/s
        return D_l * 1e-4 #mm2/us
    
    @classmethod
    def model_diffusion_transversal(cls, drift_field):
        '''
        Transversal diffusion model. Linear fit interpolation from the EXO200 paper data (Fig. 7) [
        Curve fit: m=0.010635; b=52.888942
        Linregress: m=0.013021; b=52.752949
        '''
        m=0.010635
        b=52.888942
        ans = m * drift_field + b #cm2/s
        return ans * 1e-4 #mm2/us
    
    @staticmethod
    def get_r(x,y):
        '''Get r from x and y.'''
        return np.sqrt(np.power(x,2) + np.power(y,2))
    
    @staticmethod
    def get_theta(x,y):
        '''Get theta from x and y.'''
        theta = np.arctan(y/x)
        if (x < 0) and (y < 0):
            theta = -theta
        return theta
    
    

In [3]:
class MeshGrid():
    '''
    Grid-related stuff. Construct, change, focus, etc.
    These helpstrings are becoming worse and worse, I know.
    '''
    
    def __init__(self, r_max, hex_side):
        self.r_max = r_max
        self.hex_side = hex_side
        
        self.hex_centers = construct_mesh()
        
    def construct_mesh(self):
        '''
        Construct the hexagonal gate mesh middles.
          * r_max: radius of the electrode
          * a: size of the hexagons
        '''
        a = self.hex_side # I know, I'm lazy
        r = a * np.sqrt(3)
        n_hex_x = 2 * np.ceil(75/r)
        n_hex_y = 2 * np.ceil(75/(np.sqrt(2)*a))
        
        hex_centers, _ = create_hex_grid(nx=n_hex_x, ny=n_hex_y, 
                                 min_diam = r,crop_circ = self.r_max,
                                 do_plot = False)
        
        return hex_centers
    
    def distance_to_reference_grid(self,pos_xy):
        '''
        Given an array of reference positions (hex grid in this partivular case) - format 
        [[xs,ys]] -, return the distance of a point (x0,y0) to these reference points.
        pos_xy is a 1d 2 value array.
        '''
        hex_distance = np.sqrt(np.power(self.hex_centers[:,0]-pos_xy[0],2) + 
                               np.power(self.hex_centers[:,1]-pos_xy[1],2)
                              )
        return hex_distance

    def get_closest_hex(self, pos_xy, value = True):
        '''
        Given a reference grid and a point on the plane, return the grid index where 
        the point is closeste to a grid point.
        '''
        distances = distance_to_reference_grid(self.hex_centers,pos_xy)
        idx_min_dist = np.argmin(distances)
        if value == True:
            return self.hex_centers[idx_min_dist]
        else:
            return idx_min_dist
    
    def focus_on_grid(self,pos_array):
        '''
        Focuses the electrons to the center of the hexagons. `pos_array` must
        be of the shape (N, 2), where pos_array[:,0] are x values and 
        pos_array[:,1] are y values.
        Returns a (N,2) array of the positions after the focus effect.
        '''
        pos_focus = np.apply_along_axis(self.get_closest_hex, 1, pos_array)
        return pos_focus

In [3]:
class Xelamp:
    '''
    Properties and functions of the Xe lamp and its brightly 
    shining effects on producing electrons on the photocathode.
    '''
    
    def __init__(self,delta_t_lamp):
        self.numerical_aperture = 0.22
        self.distance2photok = 2 # mm
        self.length_xy = self.numerical_aperture*self.distance
        
        #time step to consider when reconstructing the lamp pulse
        #use 2 ns (0.002 us) to match ADC freq or 0.25 for testing 
        #(from YB's original code)
        self.delta_t_lamp = delta_t_lamp 
        self.times_lamp = np.linspace(0,6,6/self.delta_t_lamp)
        
    @classmethod
    def pulse_lamp(cls,t):
        '''
        Parametrization of electrons emitted by a pulse of the lamp.
        '''
        calc = 6e4*np.exp(-(t-2.8)**2/2/(2.90/2.355)**2 )
        return calc
    
    def emitted_electrons_in_interval(self,t0,tf, error = False):
        '''
        Integrate the lamp pulse to number of electrons.
        '''
        population, error = scipy.integrate.quad(Pulse_lamp, 0,6,epsrel = 1e-6)
        if error == False:
            return population
        else:
            return population, error
    
    def init_positions(self, n_electrons, shape):
        '''
        Initial spread of electrons on x,y and z.
        Standard version uses a Gaussian spread over x and y 
        with sigma given by the lamp aperture.
        '''
        sigma_xy = self.numerical_aperture * self.distance2photok
        mu, sigma = 0, np.sqrt(sigma_xy) # mean and standard deviation
        X0 = np.random.normal(mu, sigma, n_electrons)
        Y0 = np.random.normal(mu,sigma, n_electrons)
        #Z0 = np.random.normal(mu,1e-3, n_electrons)
        Z0 = np.zeros(n_electrons)
        return X0,Y0,Z0

In [5]:
class Electron_drift:
    '''
    Processes and variables related to teh drift of electrons through the LXe TPC.
    Needs the input of a TPC object and an int n_electrons
    '''
    
    def __init__(self, TPC, n_electrons):
        self.TPC = TPC
        self.n_electrons = n_electrons
        self.e_lifetime = 2000 #us
        self.se_gain = 31 #pe/e-
        self.extract_efficiency = 1 # Electron extraction efficiency
        self.drift_velocity = TPC.drift_velocity 
        self.t = 0
        
    def apply_elifetime(self):
        '''
        Reduce the ammount of electrons due to the non-infinite e-lifetime.
        To finish: x,y,z_old are the final arrays of electrons.
        ''' 
        n_electrons = len(X_old)
        n_unlucky = round(n_electrons *(1-np.exp(-dt/self.e_lifetime)))
        unlucky_electrons_index = np.random.choice(n_unlucky,size = n_electrons, replace=False)  
        #delete from array -> probablu slow
        #X_new = np.delete(X_old, unlucky_electrons_index)
        #Y_new = np.delete(Y_old, unlucky_electrons_index)
        #Z_new = np.delete(Z_old, unlucky_electrons_index)
        
        #turn to nan the unlucky electrons
        X[unlucky_electrons_index] = np.nan
        Y[unlucky_electrons_index] = np.nan
        Z[unlucky_electrons_index] = np.nan
    
    def check_boundaries(self):
        '''
        Check if all the electrons are within the boundaries.
        If outside r>r_max -> turn to nan
        If at z=0 (gate) -> put into X/Y/Z_gas
        '''
        mask_at_gate = Z >= 0
        
        t_gas[mask_at_gate] = self.t
        
        R = get_r(X,Y)
        mask_outside_bound = R > self.TPC.radius
        X[outside_bound_mask] = np.nan
        Y[outside_bound_mask] = np.nan
        Z[outside_bound_mask] = np.nan
        
    def drift_step(self):
        pass
                   
    def extract_electrons(self):
        '''
        Apply extraction efficiency.
        '''
        n_electrons = len(X_old)
        n_unlucky = round(n_electrons * (1-np.exp(-dt/self.e_lifetime)))
        unlucky_electrons_index = np.random.choice(n_unlucky,size = n_electrons, replace=False)  
        
        X[unlucky_electrons_index] = np.nan
        Y[unlucky_electrons_index] = np.nan
        Z[unlucky_electrons_index] = np.nan